# AI_LIST

## Overview
This function interacts with an AI model to generate a list of items based on a prompt. It is especially useful for categorization, classification, and matching tasks where you need to assign items to predefined categories based on their content. The function is compatible with OpenAI-style APIs that support JSON output.

## Usage
To use the `AI_LIST` function in Excel, enter it as a formula in a cell, specifying your prompt and any optional arguments as needed:

```excel
=AI_LIST(prompt, [values], [temperature], [max_tokens], [model], [api_key], [api_url])
```
Replace each parameter with your desired value. The function returns a single-column list of items generated by the AI model.

## Parameters
| Parameter      | Type     | Required | Description                                                                                                 |
|---------------|----------|----------|-------------------------------------------------------------------------------------------------------------|
| prompt        | string   | Yes      | The instruction describing the list the AI should create.                                                    |
| values        | 2D list  | No       | Data from an Excel range to be included in the context sent to the AI.                                       |
| temperature   | float    | No       | Controls the randomness/creativity of the response (0.0 to 2.0). Lower values are more deterministic.        |
| max_tokens    | int      | No       | Maximum number of tokens (words/subwords) the AI should generate in its response (5 to 5000).               |
| model         | string   | No       | The specific AI model ID to use for the request (e.g., 'mistral-small-latest').                             |
| api_key       | string   | No       | API key for authentication. [Get a free API key from Mistral AI](https://console.mistral.ai/).              |
| api_url       | string   | No       | OpenAI-compatible API endpoint URL (e.g., https://api.mistral.ai/v1/chat/completions).                      |

## Return Value
| Return Value | Type    | Description                                                                                                    |
|--------------|---------|----------------------------------------------------------------------------------------------------------------|
| List Data    | 2D list | A single-column list of items as requested in the prompt. Returns `[ ["Error: ..."] ]` on failure.                |

## Demo
If either `api_key` or `api_url` is not provided, both will default to Boardflare demo values (`api_url`: https://llm.boardflare.com, `api_key`: your Microsoft login token if available). This only works for users logged in with a Microsoft account and provides limited free demo usage. You may obtain a free api_key for [Mistral AI](https://console.mistral.ai/) with your Microsoft account which offers more generous free usage and supports CORS.

## Limitations
- The quality of the list depends on the clarity of the prompt and the data provided.
- Large data ranges may exceed model context limits and result in truncated or incomplete responses.
- The function requires an internet connection to access the AI model.
- Model availability and output may vary depending on the provider or API changes.
- Sensitive or confidential data should not be sent to external AI services.
- `temperature` must be a float between 0 and 2 (inclusive). If not, a ValueError is raised.
- `max_tokens` must be an integer between 5 and 5000 (inclusive). If not, a ValueError is raised.
- If you hit the API rate limit for your provider, a message is returned instead of raising an exception.

## Benefits
- Automates list generation, categorization, and business analysis directly in Excel.
- Saves time and improves consistency in reporting and planning.
- Enables dynamic, context-aware lists using your own data.
- More flexible and powerful than manual or native Excel approaches for list generation and classification.

## Examples

### Generating a List of Key Performance Indicators (KPIs)
Ask for a list of KPIs relevant to a marketing department.

**Sample Input:**
```excel
=AI_LIST("List essential marketing KPIs for quarterly performance reviews")
```
**Sample Output:**
| |
|---------------------------|
| Customer Acquisition Cost |
| Conversion Rate |
| Return on Marketing Investment |
| Customer Lifetime Value |
| Website Traffic |
| Social Media Engagement |
| Email Open Rate |
| Marketing Qualified Leads |

### Creating a List of Risk Mitigation Strategies
Generate a list of risk mitigation strategies for a business project.

```excel
=AI_LIST("List effective risk mitigation strategies for enterprise software implementation")
```
**Sample Output:**
| |
|---------------------------|
| Phased Implementation Approach |
| Comprehensive Testing Protocol |
| Stakeholder Engagement Plan |
| Clear Change Management Process |
| Dedicated Support Team |
| Regular Progress Reviews |
| Vendor SLA Enforcement |
| Data Backup and Recovery Plan |

### Listing Compliance Requirements
Get a list of compliance requirements for a specific industry.

```excel
=AI_LIST("List key compliance requirements for healthcare organizations")
```
**Sample Output:**
| |
|---------------------------|
| HIPAA Privacy Rule |
| HIPAA Security Rule |
| HITECH Act Requirements |
| Medicare/Medicaid Compliance |
| Clinical Laboratory Improvement Amendments |
| Stark Law Compliance |
| Anti-Kickback Statute |
| Joint Commission Accreditation Standards |

### Using Values Parameter
Generate a list of action items based on specific meeting notes provided as values.

**Sample Values Data (Range `A1:A3`):**
| |
|-------------|
| Q1 revenue fell 5% below target |
| Customer complaints increased by 12% |
| New product launch delayed by 3 weeks |

```excel
=AI_LIST("List priority action items based on these quarterly business review notes:", A1:A3)
```
**Sample Output:**
| |
|------------------------------|
| Perform revenue gap analysis |
| Implement customer feedback process improvements |
| Review and optimize product development workflow |
| Conduct sales team performance evaluation |
| Develop enhanced customer satisfaction strategy |
| Revise product launch timeline and accountability |

### Creating a List of SMART Goals
Generate a list of SMART goals for a specific department.

```excel
=AI_LIST("List 5 SMART goals for an HR department focused on improving employee retention")
```
**Sample Output:**
| |
|-----------|
| Reduce employee turnover by 15% within the next 12 months through implementation of structured exit interviews and action plans |
| Increase employee engagement scores from 72% to 85% by Q4 through bi-monthly team building activities and feedback sessions |
| Improve benefits utilization rate from 65% to 80% within 6 months through targeted education sessions and simplified access processes |

In [5]:
import requests
import json

def ai_list(prompt, values=None, temperature=0.0, max_tokens=1000, model='mistral-small-latest', api_key=None, api_url="https://api.mistral.ai/v1/chat/completions"):
    """
    Uses AI to generate a list of items based on the prompt and optional values data.

    Args:
        prompt (str): Instruction for AI to create a list
        values (list, optional): 2D list containing additional data to append to prompt
        temperature (float, optional): Controls response creativity (0-2). Default is 0.0
        max_tokens (int, optional): Maximum tokens for response generation. Default is 1000
        model (str, optional): ID of the model to use
        api_key (str, optional): API key for authentication (e.g. for Mistral AI)
        api_url (str, optional): OpenAI compatible URL. (e.g., https://api.mistral.ai/v1/chat/completions for Mistral AI)

    Returns:
        list: 2D list representing the generated list data as a single column
    """
    # Demo fallback for Boardflare if no api_key or api_url
    if api_key is None or api_url is None:
        if 'idToken' in globals():
            api_key = globals()['idToken']
            api_url = "https://llm.boardflare.com"
        else:
            return [["Login on the Functions tab for limited demo usage, or sign up for a free Mistral AI account at https://console.mistral.ai/ and add your own api_key."]]

    # Validate temperature
    if not isinstance(temperature, (float, int)) or not (0 <= float(temperature) <= 2):
        raise ValueError("temperature must be a float between 0 and 2 (inclusive)")
    # Validate max_tokens
    if not isinstance(max_tokens, int) or not (5 <= max_tokens <= 5000):
        raise ValueError("max_tokens must be an integer between 5 and 5000 (inclusive)")

    # Construct a specific prompt for list generation
    list_prompt = f"Generate a list based on this request: {prompt}"
    if values is not None:
        values_str = "\n".join([str(item[0]) for item in values]) if len(values) > 0 and len(values[0]) > 0 else ""
        if values_str:
            list_prompt += f"\n\nUse this information to help create the list:\n{values_str}"
    list_prompt += "\nReturn ONLY a JSON object with a key 'items' whose value is a JSON array of the items for the list. "
    list_prompt += "Each item should be a single value. "
    list_prompt += "Do not include any explanatory text, just the JSON object. "
    list_prompt += 'For example: {"items": ["item1", "item2", "item3"]}'

    payload = {
        "messages": [{"role": "user", "content": list_prompt}],
        "temperature": temperature,
        "model": model,
        "max_tokens": max_tokens,
        "response_format": {"type": "json_object"}
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    try:
        response = requests.post(api_url, headers=headers, json=payload)
        if response.status_code == 429:
            return [["You have hit the rate limit for the API. Please try again later."]]
        response.raise_for_status()
        response_data = response.json()
        content = response_data["choices"][0]["message"]["content"]
        try:
            list_data = json.loads(content)
            if isinstance(list_data, dict) and "items" in list_data:
                list_data = list_data["items"]
            elif isinstance(list_data, dict):
                for key, value in list_data.items():
                    if isinstance(value, list):
                        list_data = value
                        break
            if isinstance(list_data, list):
                result = []
                for item in list_data:
                    if isinstance(item, list):
                        if len(item) >= 1:
                            result.append([str(item[0])])
                        else:
                            result.append([""])
                    else:
                        result.append([str(item)])
                return result
            else:
                return [["Error: Unable to parse response. Expected a list."]]
        except (json.JSONDecodeError, ValueError):
            return [["Error: Unable to generate list. The AI response wasn't in the expected format."]]
    except requests.exceptions.RequestException as e:
        return [["Error: API request failed.", str(e)]]

In [6]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent / "test"))
from test_utils import get_graph_token

def inject_id_token():
    token = get_graph_token()
    globals()["idToken"] = token

inject_id_token()

import pytest
import json

def not_error_result(result):
    # Returns True if result is not an error message
    if not isinstance(result, list) or not result:
        return False
    first = result[0][0] if isinstance(result[0], list) and result[0] else ""
    return not (isinstance(first, str) and first.startswith("Error:"))

def test_marketing_kpis():
    prompt = "List 4 essential marketing KPIs for quarterly performance reviews"
    result = ai_list(prompt)
    assert isinstance(result, list)
    assert len(result) == 4
    assert all(isinstance(item, list) and len(item) == 1 for item in result)
    assert not_error_result(result)

def test_risk_mitigation_strategies():
    prompt = "List 4 effective risk mitigation strategies for enterprise software implementation"
    result = ai_list(prompt)
    assert isinstance(result, list)
    assert len(result) == 4
    assert all(isinstance(item, list) and len(item) == 1 for item in result)
    assert not_error_result(result)

def test_compliance_requirements():
    prompt = "List 4 key compliance requirements for healthcare organizations"
    result = ai_list(prompt)
    assert isinstance(result, list)
    assert len(result) == 4
    assert all(isinstance(item, list) and len(item) == 1 for item in result)
    assert not_error_result(result)

def test_action_items_from_values():
    prompt = "List 4 priority action items based on these quarterly business review notes:"
    values = [
        ["Q1 revenue fell 5% below target"],
        ["Customer complaints increased by 12%"],
        ["New product launch delayed by 3 weeks"]
    ]
    result = ai_list(prompt, values=values)
    assert isinstance(result, list)
    assert len(result) == 4
    assert all(isinstance(item, list) and len(item) == 1 for item in result)
    assert not_error_result(result)

def test_smart_goals():
    prompt = "List 4 SMART goals for an HR department focused on improving employee retention"
    result = ai_list(prompt)
    assert isinstance(result, list)
    assert len(result) == 4
    assert all(isinstance(item, list) and len(item) == 1 for item in result)
    assert not_error_result(result)

ipytest.run()

........                                                                                        [100%]
======================================== warnings summary =========================================
..\..\.venv\Lib\site-packages\_pytest\config\__init__.py:1277
  c:\Users\brent\Code\python-apps\.venv\Lib\site-packages\_pytest\config\__init__.py:1277: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: anyio
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
5 passed, 1 warning in 1.54s
.                                                                                        [100%]
======================================== warnings summary =========================================
..\..\.venv\Lib\site-packages\_pytest\config\__init__.py:1277
  c:\Users\brent\Code\python-apps\.venv\Lib\site-packages\_pytest\config\__init__.py:1277: PytestAssertRewriteWarning: Module already imported so cannot be rewri

<ExitCode.OK: 0>

In [7]:
import gradio as gr

def run_ai_list(prompt, values, temperature, max_tokens, model):
    return ai_list(prompt, values=values, temperature=temperature, max_tokens=max_tokens, model=model)

examples = [
    [
        "List essential marketing KPIs for quarterly performance reviews",
        None,
        0.0,
        1000,
        "mistral-small-latest"
    ],
    [
        "List effective risk mitigation strategies for enterprise software implementation",
        None,
        0.0,
        1000,
        "mistral-small-latest"
    ],
    [
        "List key compliance requirements for healthcare organizations",
        None,
        0.0,
        1000,
        "mistral-small-latest"
    ],
    [
        "List priority action items based on these quarterly business review notes:",
        [
            ["Q1 revenue fell 5% below target"],
            ["Customer complaints increased by 12%"],
            ["New product launch delayed by 3 weeks"]
        ],
        0.0,
        1000,
        "mistral-small-latest"
    ],
    [
        "List 5 SMART goals for an HR department focused on improving employee retention",
        None,
        0.0,
        1000,
        "mistral-small-latest"
    ]
]

demo = gr.Interface(
    fn=ai_list,
    inputs=[
        gr.Textbox(label="Prompt", lines=2, value="List essential marketing KPIs for quarterly performance reviews"),
        gr.Dataframe(label="Values (optional)", type="array", headers=None, datatype="str", row_count=(1, "dynamic"), col_count=(1, "dynamic"), value=None),
        gr.Slider(0.0, 2.0, value=0.0, step=0.01, label="Temperature"),
        gr.Number(value=1000, label="Max Tokens"),
        gr.Textbox(value="mistral-small-latest", label="Model")
    ],
    outputs=gr.Dataframe(label="AI List Output"),
    examples=examples,
    description="Interact with the AI List function. Enter a prompt and, optionally, provide tabular values for the AI to use in generating a list. Adjust the temperature for creativity, set the maximum number of tokens for the response, and specify the model if desired.",
    flagging_mode="never",
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
